# Mikroskil Data Science Assignment Helper Notebook

> **Read me first:** This notebook is a teaching aid. It shows how to plan work that satisfies the rubric and scenarios without applying the steps to the confidential dataset. Replace every placeholder with your own implementation, document your choices, and do not submit this file as-is.

## How to use this template

- Skim the rubric summary to build your personal checklist.
- Use the Phase 1 and Phase 2 scaffolding to outline your own notebook.
- Each scenario playbook maps the official questions to analysis patterns; adapt them with real data.
- Replace every `raise NotImplementedError` block with working code once you understand the intent.
- Keep a decision log in the provided cell to document cleaning rules and assumptions.

## Rubric at a glance

### Phase 1: Python Structures, Gathering & Cleaning
| Criterion | Evidence to produce | Weight |
| --- | --- | --- |
| Dataset understanding | Load CSV, inspect `head()`, `info()`, and explain meaning of key columns. | 10% |
| Raw Python data structures | Lists, tuples, dicts built from a manageable subset (first 5-10 rows). | 20% |
| Numpy & pandas structures | Use `np.ndarray`, `pd.Series`, clean base `DataFrame` with correct dtypes. | 15% |
| Indexing & subsetting | Demonstrate `.loc`, `.iloc`, boolean filters, aggregations tied to scenario. | 15% |
| Data gathering / reshaping | Group, pivot, merge, create derived columns fit for scenario. | 15% |
| Data cleaning | Handle types, NaN, outliers, useless columns; justify every choice. | 20% |
| Code quality & documentation | Modular, readable code, comments, clear layout. | 5% |

### Phase 2: EDA, Visualization, Analysis
| Criterion | Evidence to produce | Weight |
| --- | --- | --- |
| EDA breadth & depth | Scenario-relevant descriptive stats, feature engineering, comparative views. | 20% |
| Visualization quality | 3-5 well-chosen plots with titles, labels, readable legends. | 25% |
| Scenario-specific metrics | Compute and interpret the required KPIs for your scenario. | 25% |
| Interpretation & insights | Write narrative conclusions, highlight limitations. | 20% |
| Communication & storytelling | Structured notebook: intro → method → results → recommendations. | 10% |

## Suggested workflow

1. Read the scenario brief carefully and highlight required metrics.
2. Sketch deliverables for Phase 1 and Phase 2 in your own words.
3. Duplicate this helper into a working notebook and remove placeholders.
4. Load the dataset, build raw Python structures, then transition to pandas.
5. Tackle cleaning tasks before deep analysis; log every decision.
6. Build EDA visuals, answer scenario questions, and draft insights.
7. Review rubric weights and verify that each item has explicit evidence.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Configure pandas display for wide tables; adjust as needed.
pd.options.display.max_columns = 100

DATA_PATH = Path("drive/MyDrive/data-science/claim_invoice_items_data.csv")
if not DATA_PATH.exists():
    raise FileNotFoundError("Update DATA_PATH so it points at the invoice-item CSV before running this helper.")

### Data loading helper

Replace the placeholder path above and wire up the loader function below. Add `dtype` hints if the CSV has scientific notation or numeric strings.

In [ ]:
def load_invoice_items(path: Path) -> pd.DataFrame:
    """Read the raw invoice-item CSV into a pandas DataFrame."""
    # TODO: adjust dtype arguments or parse_dates once you inspect the CSV header.
    df = pd.read_csv(
        path,
        parse_dates=["created_at", "updated_at", "deleted_at", "indate", "outdate"],
    )
    return df

# Uncomment after configuring DATA_PATH
raw_df = load_invoice_items(DATA_PATH)
raw_df.head()

### Decision log helper

Document every material assumption, cleaning rule, or exclusion so the instructor can follow your reasoning.

In [ ]:
decision_log = []

def log_decision(step: str, choice: str, impact: str) -> None:
    """Append a structured note to the decision log."""
    note = {"step": step, "choice": choice, "impact": impact}
    decision_log.append(note)

# Example placeholder (delete after adding your own entries)
log_decision("template", "Describe your first decision here", "Summarize the effect on the analysis")
decision_log

## Phase 1: Data structures, gathering, and cleaning

### 1. Dataset understanding (10%)

- Load the CSV, inspect `head()`, `info()`, and `describe(include="all")`.
- In a markdown cell, explain what each key column (provider, benefit_category, quantity, total_price, icd10_1, etc.) represents.
- Clarify the unit of analysis (invoice item) and the meaning of rows vs columns.

In [ ]:
def summarize_dataset(df: pd.DataFrame) -> None:
    """Template for reporting dataset shape and basic column meanings."""
    if df is None:
        raise ValueError("Pass the loaded DataFrame before calling this helper.")
    print(f"Rows: {df.shape[0]:,}  Columns: {df.shape[1]}")
    display(df.head(3))
    display(df.info())
    display(df.describe(include="all"))
    # TODO: add narrative summary in a markdown cell after running this function.

summarize_dataset(raw_df)  # Uncomment once raw_df is defined

## Data Overview
This dataset contains invoice-level healthcare billing data.
Each row represents a single invoice item, meaning one medical service, procedure, or medication billed under a particular provider’s invoice.
Multiple rows can belong to the same invoice if the patient received several services.

## Key Columns
- `id` Identifier of each item claims, can appear multiple times if claim has multiple items.
- `created_at` Timestamp when the item was captured in the source system.
- `claim_id` Claim header identifier grouping related invoice items.
- `item_label` Invoice line classification (charge, discount, cancel, placeholder)
- `item_type` Source line type.
- `matched_name` System data that matched with the line item on the invoice.
- `quantity` Quantity billed for the line item.
- `tarif_description` Nearest description for hospital charge tariffs item found in tarif reference database
- `tarif_price` Tariff amount tied to `tarif_description`.
- `total_price` Total billed amount for the item.

## Row and Column Representation
- Each row represents 1 invoice item (service or medicine product)
- Each column represents attribute of that item, such as `item_type`, `quantity`, `tarif_price`, `total_price`, etc.

### 2. Raw Python data structures (20%)

- Work with a small slice: `sample_records = df.head(10).to_dict(orient="records")`.
- Build lists (e.g., provider names), tuples (immutable identifiers), and dicts (e.g., code → description).
- Explain in text why each structure suits the scenario.

In [ ]:
def build_python_structures(sample_records: list) -> dict:
    """Return example Python structures built from a small subset of rows."""
    if not sample_records:
        raise ValueError("Provide 5-10 records from your DataFrame.")

    providers = [row["provider_name"] for row in sample_records if "provider_name" in row]
    item_keys = [(row.get("id"), row.get("description")) for row in sample_records]
    provider_map = {row.get("provider_code"): row.get("provider_name") for row in sample_records}

    structures = {
        "providers_list": providers,
        "item_keys_tuple": item_keys[:3],
        "provider_lookup": provider_map,
    }
    return structures

example_structures = build_python_structures(raw_df.head(10).to_dict(orient="records"))
display(example_structures)

## Why each structure suits the screnario?
1. `provider_list` will help to determine which provider contributes the highest total claim cost?
2. `item_keys_tuple` will show which items or drug types contribute most to the claim cost
3. `provider_lookup` will simplifies grouping and joining for cost summaries

### 3. Numpy and pandas structures (15%)

- Convert relevant numeric columns to `np.ndarray` and `pd.Series`.
- Demonstrate vectorized operations (e.g., `df["unit_price"].to_numpy()` for calculations).
- After enforcing dtypes, create a clean working copy `clean_df = df.copy()` for downstream work.

In [3]:
def enforce_dtypes(df: pd.DataFrame, numeric_cols: list, date_cols: list) -> pd.DataFrame:
    """Create a cleaned copy with enforced numeric and datetime types."""
    clean_df = df.copy()
    for col in numeric_cols:
        clean_df[col] = pd.to_numeric(clean_df[col], errors="coerce")
    for col in date_cols:
        clean_df[col] = pd.to_datetime(clean_df[col], errors="coerce")

    # Vectorized numeric calculation
    clean_df["computed_total"] = clean_df["quantity"] * clean_df["unit_price"]

    # Vectorized date calculation
    clean_df["los_days"] = (clean_df["outdate"] - clean_df["indate"]).dt.days

    return clean_df

numeric_columns = ["quantity", "unit_price", "total_price"]
date_columns = ["created_at", "updated_at", "deleted_at", "indate", "outdate"]
clean_df = enforce_dtypes(raw_df, numeric_columns, date_columns)
clean_df.head()

NameError: name 'pd' is not defined

After listing the numeric colums we can answer the first question inside the given rubric. " Create a list of all numeric columns and display their min, max, and mean values. "

In [ ]:
def list_numeric_columns_and_stats(df: pd.DataFrame) -> pd.DataFrame:
    """
    1) List numeric columns and compute min, max, mean.
    Returns a DataFrame indexed by column name with columns: dtype, min, max, mean, non_null_count.
    """
    numeric_cols = numeric_columns
    stats = []
    for col in numeric_cols:
        col_ser = df[col].dropna()
        stats.append(
            {
                "column": col,
                "min": col_ser.min() if not col_ser.empty else np.nan,
                "max": col_ser.max() if not col_ser.empty else np.nan,
                "mean": col_ser.mean() if not col_ser.empty else np.nan,
            }
        )
    stats_df = pd.DataFrame(stats).set_index("column")
    return stats_df

I converted numeric columns like quantity, unit_price, and total_price into both NumPy arrays and pandas Series to demonstrate vectorized computation.
NumPy arrays are efficient for numerical operations, while pandas Series retain column labels and indexes, making them easier to align with other data in the DataFrame.

### 4. Indexing and subsetting (15%)

- Prepare `.loc` and `.iloc` examples tied to your scenario filters.
- Demonstrate boolean filters (e.g., `clean_df[clean_df["benefit_category"] == "medicine"]`).
- Store at least one filtered subset for later EDA.

In [ ]:
def demo_indexing(df: pd.DataFrame) -> None:
    """Showcase loc, iloc, and boolean indexing for rubric evidence."""
    # TODO: adapt column names and filters to your scenario.
    medicine_mask = df["benefit_category"].eq("medicine")
    sample_loc = df.loc[medicine_mask, ["provider_name", "quantity", "total_price"]].head()
    sample_iloc = df.iloc[:5, :5]
    display(sample_loc)
    display(sample_iloc)

# demo_indexing(clean_df)

### 5. Data gathering and reshaping (15%)

- Use `groupby`, `pivot_table`, or joins to engineer the structures your scenario needs.
- Create at least one derived column (e.g., `computed_total = quantity * unit_price`).
- If merging external mappings (ICD groupings, region labels), show the helper dict or `Series`.

In [ ]:
def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add derived columns expected by multiple scenarios."""
    engineered = df.copy()
    if {"quantity", "unit_price"}.issubset(engineered.columns):
        engineered["computed_total"] = engineered["quantity"] * engineered["unit_price"]
    if {"indate", "outdate"}.issubset(engineered.columns):
        engineered["los_days"] = (engineered["outdate"] - engineered["indate"]).dt.days
    return engineered

# engineered_df = engineer_features(clean_df)

### 6. Data cleaning (20%)

- Detect missing values with `df.isna().sum()` and describe treatment strategies.
- Address negatives or impossible values (drop, cap, absolute) and justify in the decision log.
- Drop low-value columns (`deleted_at`, `estimated_pack_prices`) with a note explaining why.

In [ ]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """Template for handling missingness, negatives, and redundant columns."""
    cleaned = df.copy()
    summary_missing = cleaned.isna().sum()
    print("Missing values summary (top 10):")
    display(summary_missing.head(10))

    # TODO: implement scenario-appropriate rules below.
    # Example placeholders: remove columns, flag negatives, fill nulls.
    # cleaned = cleaned.drop(columns=["deleted_at", "estimated_pack_prices"], errors="ignore")
    # cleaned.loc[cleaned["quantity"] < 0, "quantity_flag"] = "negative"

    raise NotImplementedError("Replace placeholder rules with your cleaning logic and document decisions.")

# final_df = clean_data(engineered_df)

### 7. Code quality and documentation (5%)

- Refactor repetitive tasks into functions (as scaffolded above).
- Add short comments before non-obvious logic.
- Use markdown headings to organize narrative: context, methods, results, insights.
- Double-check notebook execution order before submission.

## Phase 2: EDA, visualization, and interpretation

### EDA toolkit

Use the helpers below as starting points. Swap in scenario-specific filters or columns.

In [ ]:
def describe_numeric(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    """Return descriptive statistics for selected numeric columns."""
    stats = df[columns].describe().T
    stats["missing"] = df[columns].isna().sum()
    return stats


def plot_distribution(df: pd.DataFrame, column: str, kind: str = "hist") -> None:
    """Plot a basic distribution for the chosen column."""
    if kind == "hist":
        sns.histplot(df[column].dropna(), kde=False)
    elif kind == "box":
        sns.boxplot(x=df[column].dropna())
    else:
        raise ValueError("kind must be 'hist' or 'box'")
    plt.title(f"Distribution of {column}")
    plt.show()


def plot_grouped_bar(df: pd.DataFrame, index_col: str, value_col: str, top_n: int = 10) -> None:
    """Plot top N categories by aggregated value."""
    summary = (
        df.groupby(index_col)[value_col]
        .sum()
        .sort_values(ascending=False)
        .head(top_n)
    )
    summary.plot(kind="bar")
    plt.ylabel(value_col)
    plt.title(f"Top {top_n} {index_col} by {value_col}")
    plt.xticks(rotation=45, ha="right")
    plt.show()

### Insights narrative

- After generating visuals, write 2-3 paragraphs connecting findings to the business question.
- Call out limitations, data quirks, or assumptions (e.g., synthetic sample, single-claim scope).
- End with actionable recommendations or next steps.

## Scenario playbooks

Each playbook summarizes the official brief. Follow the steps using your cleaned DataFrame. The goal is to show your reasoning path, not to memorize outputs.

### Scenario 1 – Pharmacy Cost Optimization

**Role reminder:** Pharmacy Cost Analyst focusing on medicine items.

**Rubric emphasis:** Negative quantity/price handling, reconstructed totals, scenario-tailored metrics.

**Recommended flow:**
1. Filter to `benefit_category == "medicine"` and `sub_benefit_name == "Obat-Obatan"`.
2. Build raw Python structures from the filtered subset (list of `chemical_substance`, tuple `(kfa_code, generic_name)`, dict `item_label -> chemical_substance`).
3. Convert cost fields to numeric, flag negatives, create `computed_total`, and log decisions.
4. Aggregate cost by item, provider, and province; compute contribution percentages.
5. Plot distributions (histogram/boxplot) and top cost drivers.
6. Summarize anomalies and draft 3-5 recommendations.

**Answering the official questions:**
1. Use raw Python list comprehension over the filtered subset to capture unique `chemical_substance`.
2. Create tuples for immutable identifiers and explain immutability in a markdown cell.
3. Build dict via `{row['item_label']: row['chemical_substance'] for row in subset}` and count entries.
4. After loading data, print `df.shape` and interpret rows vs columns.
5. Cast numeric columns and note parsing issues (e.g., commas, strings).
6. Locate negative values with boolean filters and explain treatment.
7. Compare `computed_total` and `total_price` via a new column or `.assign()`.
8. Justify column drops (e.g., `estimated_pack_prices`).
9. Demonstrate `.loc` for provider/province slices.
10. Conclude with a markdown summary of data quality findings.

### Scenario 2 – Length of Stay & Inpatient Tariff Consistency

**Role reminder:** Hospital Network Performance Analyst.

**Rubric emphasis:** Accurate LOS calculation, tariff handling, date cleaning.

**Recommended flow:**
1. Parse `indate` and `outdate` as datetime and compute `los_days`.
2. Store first 10 LOS values in a list; create tuples `(card_number, indate, outdate)`.
3. Build provider metadata dict with nested attributes.
4. Handle invalid or negative LOS, missing tarif values, and document rationale.
5. Create summary tables for tariff by provider and LOS buckets.
6. Visualize LOS distributions and tariff vs LOS relationships.

**Answering the official questions:**
- Use `.head()` plus `.apply()` to compute the LOS list.
- Explain tuple immutability for patient stay IDs in markdown.
- Construct provider dict via `.drop_duplicates()` then `.set_index().to_dict(orient="index")`.
- Count invalid dates with `df['indate'].isna().sum()` etc.
- Filter rows with negative LOS and describe fixes.
- Convert `tarif_price` to numeric, treat `tarif_description == '-'` as missing.
- Produce average tariff per provider with `groupby`.
- Showcase indexing for `los_days > 5`.
- Document every cleaning step in a dedicated markdown section.

### Scenario 3 – Benefit Category Utilization Across Countries

**Role reminder:** Benefit Design Analyst comparing benefit mix across provinces.

**Rubric emphasis:** Accurate aggregation by benefit dimensions and comparative visuals.

**Recommended flow:**
1. List unique `benefit_category` values and create tuples `(benefit_category, sub_benefit_name)`.
2. Map province to benefit categories using dicts or `defaultdict(list)`.
3. Clean cost columns, drop irrelevant fields, and create aggregated tables.
4. Produce stacked/clustered bars, pies, and heatmaps showing cost distribution.
5. Interpret which provinces or benefit categories drive costs and why.

**Answering the official questions:**
- Use Python list comprehension for unique benefit categories.
- Demonstrate tuple creation and show sample output in markdown.
- Build province → benefits dict via groupby and `.unique()`.
- Report data type checks and missingness for benefit columns.
- Aggregate cost by benefit and province for metrics and visuals.
- Provide narrative on cost shares, outliers, correlations, and recommendations.

### Scenario 4 – ICD-10 Case Mix Profiling

**Role reminder:** Clinical Data Analyst profiling diagnoses and costs.

**Rubric emphasis:** ICD grouping dict, missingness analysis, clinical insights.

**Recommended flow:**
1. Inspect ICD columns for missingness and convert to string.
2. Create list of frequent `icd10_1` values and tuples `(icd10_1, description_1)`.
3. Build a manual ICD grouping dict and apply it safely.
4. Engineer `diagnosis_count` from non-null ICD columns.
5. Aggregate counts and costs by ICD code and group; visualize distributions.
6. Compare costs by diagnosis count and discuss clinical insights with limitations.

**Answering the official questions:**
- Use `.value_counts().head(10)` for top ICD codes.
- Pair codes and descriptions in tuples and explain usage.
- Count missing values and describe handling strategy.
- Plot top ICDs and cost distributions using bar/box charts.
- Discuss anomalies, multi-diagnosis patterns, and future data needs.

### Scenario 5 – Member-Level Exposure & Cost per Member

**Role reminder:** Member Analytics Data Scientist aggregating items to members.

**Rubric emphasis:** Correct member-level aggregation, segmentation by demographics.

**Recommended flow:**
1. Extract unique member identifiers and confirm no duplicates.
2. Build tuples `(card_number, member_name)` and dict `card_number -> sex`.
3. Aggregate to member level: total cost, item count, distinct providers, average unit price.
4. Investigate missing or inconsistent member data, document assumptions (e.g., `dob` as year).
5. Visualize distributions and compare metrics by sex; identify outliers.
6. Produce recommendations for underwriting or care management.

**Answering the official questions:**
- Use `.unique()` for card numbers and `.duplicated()` to check inconsistencies.
- Build per-member summary with `groupby('card_number').agg(...)`.
- Filter and visualize outliers, scatter item count vs cost, boxplots by sex.
- Discuss utilization patterns and actionable insights.

### Scenario 6 – Fraud, Waste, & Abuse Screening

**Role reminder:** Special Investigations Unit analyst flagging anomalies.

**Rubric emphasis:** Strong cleaning, anomaly rules, interpretable watch list.

**Recommended flow:**
1. Identify rows with `quantity <= 0` or `total_price <= 0`; store in a list.
2. Create tuples `(provider_code, benefit_category, description)` for suspicious items.
3. Build dict `provider_code -> suspicious_count`.
4. Define outlier rule for `unit_price` (IQR or percentile) and create `is_suspicious`.
5. Visualize distributions and provider counts; compute cost share of suspicious rows.
6. Summarize watch list and limitations (signals, not proof).

**Answering the official questions:**
- Use boolean indexing to list suspicious rows.
- Count anomalies per provider with `groupby`.
- Plot boxplots and scatterplots highlighting suspicious points.
- Quantify cost share and document reasoning.

### Scenario 7 – Provider Segmentation & Service Mix

**Role reminder:** Provider Network Analyst segmenting providers.

**Rubric emphasis:** Provider-level aggregation, segmentation logic, clear visuals.

**Recommended flow:**
1. List unique providers and confirm metadata consistency.
2. Create tuples `(provider_code, provider_name, province)` and dictionaries summarizing cost by category.
3. Aggregate metrics: total cost, item counts, member counts, cost share by benefit.
4. Define segmentation rules (e.g., >60% medicine cost → medication-heavy).
5. Visualize stacked bars and scatterplots by segment.
6. Interpret segment traits and propose actions.

**Answering the official questions:**
- Compute cost shares, classify providers, and count membership per segment.
- Plot stacked bars and scatter of total cost vs members colored by segment.
- Highlight top cost providers and discuss regional differences.

### Scenario 8 – Claim Cost Prediction & Feature Insight

**Role reminder:** Data analyst supporting predictive modeling.

**Rubric emphasis:** Comprehensive cleaning, feature engineering, insight into predictors.

**Recommended flow:**
1. Inventory numeric columns, data types, and missingness.
2. Drop redundant columns, enforce numeric and datetime types.
3. Engineer features: `claim_total_cost`, `benefit_diversity`, `los_days`, item counts.
4. Validate aggregations (no data loss) and build correlation visuals.
5. Identify key drivers, outliers, and recommend modeling features.
6. Write a readiness narrative summarizing data quality and insights.

**Answering the official questions:**
- Use descriptive stats, missingness dictionary, and column profiling.
- Create aggregation pipelines for claim-level metrics.
- Plot distributions, heatmaps, scatterplots, and interpret correlations.
- Prioritize predictive features and articulate business implications.

## Final checklist before submission

- [ ] Every Phase 1 rubric item has visible evidence (code + explanation).
- [ ] Every Phase 2 rubric item has relevant plots and written interpretation.
- [ ] Scenario-specific questions are answered in code and markdown, referencing figures.
- [ ] Decision log is complete and reflects cleaning choices.
- [ ] Notebook runs top-to-bottom without placeholder errors.
- [ ] Conclusions include limitations and next-step ideas.